In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import torch

sys.path.append('..')
from src.utils import *
from src.models import *
from src.configs import *


from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
rcParams.update(fig_params)

import warnings

%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

## Set device

In [2]:
DEVICE = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print('Using device:', DEVICE)

Using device: cuda:3


## Load experiments file

In [3]:
FILENAME = 'Experiments_MASSIVE.xlsx'

In [4]:
exp_df = pd.read_excel(os.path.join('..', 'experiments', FILENAME), dtype=FORMAT)
exp_df.head()

,exp_id,seed,torch_dtype,samples_number,features_number,snr_db,informative_frac,folds_number,initialization,epochs_number,learning_rate
0,MASSIVE000,8925,64,1000,100,20,0.5,5,1.0,300,350.0
1,MASSIVE001,65459,64,1000,100,20,0.5,5,1.0,300,350.0
2,MASSIVE002,30300,64,1000,100,20,0.5,5,1.0,300,350.0
3,MASSIVE003,38747,64,1000,100,20,0.5,5,1.0,300,350.0
4,MASSIVE004,26932,64,1000,100,20,0.5,5,1.0,300,350.0


## Run experiments

In [5]:
exp_df = exp_df.iloc[100:].copy()
exp_df

,exp_id,seed,torch_dtype,samples_number,features_number,snr_db,informative_frac,folds_number,initialization,epochs_number,learning_rate
100,MASSIVE100,47464,64,1000,1100,20,0.5,5,1.0,300,350.0
101,MASSIVE101,57137,64,1000,1100,20,0.5,5,1.0,300,350.0
102,MASSIVE102,41965,64,1000,1100,20,0.5,5,1.0,300,350.0
103,MASSIVE103,12476,64,1000,1100,20,0.5,5,1.0,300,350.0
104,MASSIVE104,57991,64,1000,1100,20,0.5,5,1.0,300,350.0
105,MASSIVE105,32831,64,1000,1100,20,0.5,5,1.0,300,350.0
106,MASSIVE106,43424,64,1000,1100,20,0.5,5,1.0,300,350.0
107,MASSIVE107,91048,64,1000,1100,20,0.5,5,1.0,300,350.0
108,MASSIVE108,35536,64,1000,1100,20,0.5,5,1.0,300,350.0
109,MASSIVE109,99629,64,1000,1100,20,0.5,5,1.0,300,350.0


In [ ]:
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    os.environ["PYTHONWARNINGS"] = 'ignore'

for i, exp in exp_df.iterrows():
    EXP_ID = exp.exp_id
    SEED = exp.seed
    if exp.torch_dtype == 64:
        DTYPE = torch.float64
    elif exp.torch_dtype == 32:
        DTYPE = torch.float32
    N = exp.samples_number
    D = exp.features_number
    SNR_DB = exp.snr_db
    INFORMATIVE_FRAC = exp.informative_frac
    NUM_FOLDS = exp.folds_number
    INITIALIZATION = exp.initialization
    EPOCHS = exp.epochs_number
    LEARNING_RATE = exp.learning_rate
    
    ### Create output folders
    os.makedirs(os.path.join('..', 'results', 'multiridge', FILENAME.split('.')[0], 'data'), exist_ok=True)
    data_folder = os.path.join('..', 'results', 'multiridge', FILENAME.split('.')[0], 'data')
    os.makedirs(os.path.join('..', 'results', 'multiridge', FILENAME.split('.')[0], 'images'), exist_ok=True)
    images_folder = os.path.join('..', 'results', 'multiridge', FILENAME.split('.')[0], 'images')
    
    ### Data generation
    train, test, features, target, theta_true = generate_synthetic_dataset(n_samples_train=N, n_samples_test=100000, n_features=D, n_informative=int(D*INFORMATIVE_FRAC),
                                                                           snr_db=SNR_DB, random_state=SEED)
    check_snr(train, test, features, target, theta_true, snr_db=SNR_DB)

    ### MultiRidge optimization
    initialization = np.ones(D)*exp.initialization
    learning_rate = LinearLR(initial_lr=LEARNING_RATE, decay=0.999)
    model = MultiRidge(lambda_vector=initialization, folds=NUM_FOLDS, shuffle=True, random_state=SEED, normalize=True,
                       epochs=EPOCHS, learning_rate=learning_rate, scoring={'r2': r2_score, 'mse': mean_squared_error},
                       verbose=10, device=DEVICE, dtype=DTYPE)
    model.fit(train[features], train[target], eval_set=(test[features], test[target]))
    results_dict = {'exp_id': EXP_ID, 'history_df': model.history_df}#, 'history_coef': history_coef}
    with open(os.path.join(data_folder, f'{EXP_ID}.pickle'), 'wb') as handle:
        pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    ### Plot dashboard
    pathfile_mr = os.path.join(images_folder, f'{EXP_ID}')
    fig_params = {'train_color': "#9fc377", 'test_color': "#0272a2", 'valid_color': "#ca0b03"}
    metric_params = {'metric': 'r2', 'metric_func': r2_score, 'metric_label': '$R^2$'}
    plot_monitoring_dashboard_multiridge(train, test, features, target, folds=NUM_FOLDS, model=model, history=model.history_df,
                                         fig_params=fig_params, metric_params=metric_params, savefig=pathfile_mr, show=False)

Epoch 31: Train r2: 0.990, Valid r2: 0.925, Test r2: 0.923, Train mse: 0.010, Valid mse: 0.074, Test mse: 0.077
Epoch 41: Train r2: 0.987, Valid r2: 0.906, Test r2: 0.919, Train mse: 0.013, Valid mse: 0.094, Test mse: 0.081
Epoch 51: Train r2: 0.990, Valid r2: 0.931, Test r2: 0.927, Train mse: 0.010, Valid mse: 0.068, Test mse: 0.073
Epoch 61: Train r2: 0.992, Valid r2: 0.941, Test r2: 0.939, Train mse: 0.008, Valid mse: 0.058, Test mse: 0.062
Epoch 71: Train r2: 0.991, Valid r2: 0.932, Test r2: 0.939, Train mse: 0.009, Valid mse: 0.068, Test mse: 0.062
Epoch 81: Train r2: 0.990, Valid r2: 0.926, Test r2: 0.936, Train mse: 0.010, Valid mse: 0.073, Test mse: 0.064
Epoch 91: Train r2: 0.991, Valid r2: 0.927, Test r2: 0.936, Train mse: 0.009, Valid mse: 0.072, Test mse: 0.064
Epoch 101: Train r2: 0.991, Valid r2: 0.929, Test r2: 0.944, Train mse: 0.009, Valid mse: 0.072, Test mse: 0.057
Epoch 111: Train r2: 0.993, Valid r2: 0.947, Test r2: 0.949, Train mse: 0.007, Valid mse: 0.052, Test m

  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 1: Train r2: 0.856, Valid r2: 0.483, Test r2: 0.528, Train mse: 0.144, Valid mse: 0.518, Test mse: 0.469
Epoch 11: Train r2: 0.988, Valid r2: 0.914, Test r2: 0.891, Train mse: 0.012, Valid mse: 0.085, Test mse: 0.108
Epoch 21: Train r2: 0.990, Valid r2: 0.923, Test r2: 0.916, Train mse: 0.010, Valid mse: 0.076, Test mse: 0.083
Epoch 31: Train r2: 0.990, Valid r2: 0.916, Test r2: 0.925, Train mse: 0.010, Valid mse: 0.083, Test mse: 0.075
Epoch 41: Train r2: 0.990, Valid r2: 0.931, Test r2: 0.930, Train mse: 0.010, Valid mse: 0.069, Test mse: 0.070
Epoch 51: Train r2: 0.991, Valid r2: 0.937, Test r2: 0.938, Train mse: 0.009, Valid mse: 0.062, Test mse: 0.062
Epoch 61: Train r2: 0.990, Valid r2: 0.928, Test r2: 0.931, Train mse: 0.010, Valid mse: 0.071, Test mse: 0.069
Epoch 71: Train r2: 0.992, Valid r2: 0.946, Test r2: 0.943, Train mse: 0.008, Valid mse: 0.053, Test mse: 0.057
Epoch 81: Train r2: 0.991, Valid r2: 0.932, Test r2: 0.933, Train mse: 0.009, Valid mse: 0.067, Test mse:

  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 1: Train r2: 0.868, Valid r2: 0.518, Test r2: 0.556, Train mse: 0.132, Valid mse: 0.482, Test mse: 0.397
Epoch 11: Train r2: 0.990, Valid r2: 0.929, Test r2: 0.889, Train mse: 0.010, Valid mse: 0.069, Test mse: 0.099
Epoch 21: Train r2: 0.991, Valid r2: 0.930, Test r2: 0.913, Train mse: 0.009, Valid mse: 0.070, Test mse: 0.078
Epoch 31: Train r2: 0.995, Valid r2: 0.958, Test r2: 0.939, Train mse: 0.005, Valid mse: 0.042, Test mse: 0.054
Epoch 41: Train r2: 0.992, Valid r2: 0.939, Test r2: 0.936, Train mse: 0.008, Valid mse: 0.061, Test mse: 0.058
Epoch 51: Train r2: 0.994, Valid r2: 0.950, Test r2: 0.936, Train mse: 0.006, Valid mse: 0.050, Test mse: 0.057
Epoch 61: Train r2: 0.992, Valid r2: 0.939, Test r2: 0.937, Train mse: 0.008, Valid mse: 0.061, Test mse: 0.056
Epoch 71: Train r2: 0.996, Valid r2: 0.969, Test r2: 0.949, Train mse: 0.004, Valid mse: 0.030, Test mse: 0.046
Epoch 81: Train r2: 0.992, Valid r2: 0.927, Test r2: 0.929, Train mse: 0.008, Valid mse: 0.073, Test mse:

  0%|          | 0/300 [00:00<?, ?it/s]

Epoch 1: Train r2: 0.852, Valid r2: 0.461, Test r2: 0.516, Train mse: 0.148, Valid mse: 0.537, Test mse: 0.498
Epoch 11: Train r2: 0.985, Valid r2: 0.899, Test r2: 0.852, Train mse: 0.015, Valid mse: 0.099, Test mse: 0.152
Epoch 21: Train r2: 0.987, Valid r2: 0.909, Test r2: 0.891, Train mse: 0.013, Valid mse: 0.089, Test mse: 0.112
Epoch 31: Train r2: 0.986, Valid r2: 0.902, Test r2: 0.899, Train mse: 0.014, Valid mse: 0.096, Test mse: 0.104
Epoch 41: Train r2: 0.990, Valid r2: 0.931, Test r2: 0.917, Train mse: 0.010, Valid mse: 0.067, Test mse: 0.086
Epoch 51: Train r2: 0.989, Valid r2: 0.918, Test r2: 0.914, Train mse: 0.011, Valid mse: 0.082, Test mse: 0.089
Epoch 61: Train r2: 0.988, Valid r2: 0.913, Test r2: 0.920, Train mse: 0.012, Valid mse: 0.087, Test mse: 0.082
Epoch 71: Train r2: 0.988, Valid r2: 0.922, Test r2: 0.930, Train mse: 0.012, Valid mse: 0.078, Test mse: 0.072
Epoch 81: Train r2: 0.991, Valid r2: 0.937, Test r2: 0.933, Train mse: 0.009, Valid mse: 0.063, Test mse: